## Radargestützte Niederschlagsschätzung: Fallstudie Berlin, 12.7.2018

### Umgebung einrichten

In [1]:
import wradlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial import KDTree
import helpers
import datetime as dt

In [2]:
%matplotlib inline

### Konfiguration

In [3]:
tstart = "2018-07-11 23:00:00"
tend = "2018-07-12 23:00:00"
dxsrc = "dx/raa00-dx_10392-%s-pro---bin"
dtimes = wradlib.util.from_to(tstart, tend, 300)

In [4]:
site = (helpers.radars["pro"]["lon"], helpers.radars["pro"]["lat"], helpers.radars["pro"]["alt"])
r = helpers.specs_dx["r"]
az = helpers.specs_dx["az"]

In [5]:
# http://www.stadtentwicklung.berlin.de/service/gesetzestexte/de/download/geoinformation/koordinatenreferenzsysteme.pdf
proj = wradlib.georef.epsg_to_osr(25833)

In [6]:
dataset, inLayer = wradlib.io.open_vector("shapes/RBS_OD_ORT_1412.shp")
borders, keys = wradlib.georef.get_vector_coordinates(inLayer)
bbox = inLayer.GetExtent()

### Niederschlagsschreiber laden

In [7]:
gauges = pd.read_csv("berlin_gauges.csv", sep=";", thousands=r',', encoding="cp1252", keep_default_na=False)
gdata = pd.read_csv("berlin_gages_timeseries.csv", sep=";", header=None)

In [8]:
gauges.columns = ["name", "x", "y", "veg", "flag"]
gdata.columns = ["dtime"] + [i for i in range(len(gauges))]

In [9]:
gdata.dtime = pd.to_datetime(gdata.dtime, format="%d/%m/%Y %H:%M:%S")
gdata.dtime = gdata.dtime - dt.timedelta(seconds=3600*2)
gdata = gdata.set_index("dtime")

In [10]:
gageprec = gdata.loc[tend] - gdata.loc[tstart]

KeyError: 'the label [2018-07-12 23:00:00] is not in the [index]'

### Radardaten laden: DX-Produkt (lokal, Reflektivität)

In [ ]:
dbz = np.zeros((len(dtimes), len(az), len(r))) * np.nan
for i, dtime in enumerate(dtimes):
    fpath = dxsrc % dtime.strftime("%y%m%d%H%M")
    try:
        data, meta = wradlib.io.read_dx(fpath)
        print(".", end="")
    except:
        print("Could not read: " % fpath)
    dbz[i] = data
print("")

### Niederschlag aus Reflektivität schätzen: mit und ohne Dämpfungskorrektur

In [ ]:
prec = helpers.dbz2depth(dbz)

In [ ]:
pia = wradlib.atten.correct_attenuation_constrained(dbz, a_max=1.67e-4, a_min=2.33e-5, n_a=100,
                                                    b_max=0.7, b_min=0.65, n_b=6, gate_length=1.,
                                                    constraints=[wradlib.atten.constraint_dbz,
                                                                 wradlib.atten.constraint_pia],
                                                    constraint_args=[[59.0], [20.0]])

In [ ]:
att_corr = dbz + pia
prec2 = helpers.dbz2depth(att_corr)

### Radardaten georeferenzieren

In [ ]:
coord = wradlib.georef.sweep_centroids(len(az), 1000., len(r), 0.5)
xyz = wradlib.georef.spherical_to_proj(coord[..., 0],
                                       np.degrees(coord[..., 1]),
                                       coord[..., 2], sitecoords=site, proj=proj)

### Diagnostischer Plot 1: Niederschlagsschreiber über PPI

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111, aspect="equal")
ax, pm = wradlib.vis.plot_ppi(prec.sum(axis=0), r=r, az=az, site=site, 
                              proj=proj, cmap=plt.cm.nipy_spectral, ax=ax, vmax=80)
cb = plt.colorbar(pm, shrink=0.5)
wradlib.vis.add_lines(ax, borders, color='black', lw=1)
for i in range(len(gauges)):
    color = "red"
    if gauges.veg[i]=="geprüft":
        color="white"
    plt.text(gauges.x[i], gauges.y[i], "%d" % gageprec[i],
             horizontalalignment='center', verticalalignment='center', color=color, fontsize=14)
plt.grid()
plt.xlabel("UTM Easting")
plt.ylabel("UTM Northing")
plt.xlim(bbox[0], bbox[1])
plt.ylim(bbox[2], bbox[3])

### Radarniederschlag an den Niederschlagsschreibern extrahieren

In [ ]:
# compute the KDTree
tree = KDTree(list(zip(xyz[...,0].ravel(), xyz[...,1].ravel())))
# query the tree for nearest neighbours
distances, dxix = tree.query(list(zip(gauges.x, gauges.y)), k=9)

In [ ]:
radarprec = np.median([prec.sum(axis=0).ravel()[ix_] for ix_ in dxix], axis=1)
radarprec2 = np.median([prec2.sum(axis=0).ravel()[ix_] for ix_ in dxix], axis=1)

### Diagnostischer Plot 2: Niederschlagsschreiber über PPI

In [ ]:
maxval = 80
goodix = np.array(gauges.veg=="geprüft")
fig = plt.figure(figsize=(12,6))
ax = plt.subplot(121, aspect="equal")
plt.plot([-10, 100], [-10, 100], "k--")
plt.plot(gageprec, radarprec, "k+")
plt.plot(gageprec[goodix], radarprec[goodix], "g+")
plt.xlim(0,maxval)
plt.ylim(0,maxval)
plt.xlabel("Gages (mm)")
plt.ylabel("Radar (mm)")
plt.grid()

ax = plt.subplot(122, aspect="equal")
plt.plot([-10, 100], [-10, 100], "k--")
plt.plot(gageprec, radarprec2, "k+")
plt.plot(gageprec[goodix], radarprec2[goodix], "g+")
plt.xlim(0,maxval)
plt.ylim(0,maxval)
plt.xlabel("Gages (mm)")
plt.ylabel("Radar (mm)")
plt.grid()

### RADOLAN RW-Produkt (stündlich, angeeicht) laden

In [ ]:
tstart_rw = dtimes[0].replace(minute=0) - dt.timedelta(seconds=10*60)
tend_rw = dtimes[-1].replace(minute=0) + dt.timedelta(seconds=10*60)
dtimes_rw = wradlib.util.from_to(tstart_rw, tend_rw, 3600)
rw = np.zeros((len(dtimes_rw), 900, 900)) * np.nan
rwsrc = "rw/raa01-rw_10000-%s-dwd---bin.gz"

In [ ]:
for i, dtime in enumerate(dtimes_rw):
    fpath = rwsrc % dtime.strftime("%y%m%d%H%M")
    try:
        data, meta = wradlib.io.read_radolan_composite(fpath, missing=np.nan)
        print(".", end="")
    except:
        print("Could not read: " % fpath)
    rw[i] = data
print("")

### Komposit georeferenzieren

In [ ]:
# Get coordinates
rwcoords = wradlib.georef.get_radolan_grid(900,900)
xrw, yrw = wradlib.georef.reproject(rwcoords[:,:,0], rwcoords[:,:,1],
                                   projection_source=wradlib.georef.create_osr("dwd-radolan"),
                                   projection_target=proj)

In [ ]:
fig = plt.figure(figsize=(12,6))

ax = fig.add_subplot(131, aspect="equal")
ax, pm = wradlib.vis.plot_ppi(prec.sum(axis=0), r=r, az=az, site=site, 
                              proj=proj, cmap=plt.cm.nipy_spectral, ax=ax, vmax=80)
#cb = plt.colorbar(pm, shrink=0.5)
wradlib.vis.add_lines(ax, borders, color='black', lw=0.3)
#for i in range(len(gauges)):
#    color = "red"
#    if gauges.veg[i]=="geprüft":
#        color="white"
#    plt.text(gauges.x[i], gauges.y[i], "%d" % gageprec[i],
#             horizontalalignment='center', verticalalignment='center', color=color, fontsize=12)
plt.grid()
plt.xlabel("UTM Easting")
plt.ylabel("UTM Northing")
plt.xlim(bbox[0], bbox[1])
plt.ylim(bbox[2], bbox[3])

ax = fig.add_subplot(132, aspect="equal")
ax, pm = wradlib.vis.plot_ppi(prec2.sum(axis=0), r=r, az=az, site=site, 
                              proj=proj, cmap=plt.cm.nipy_spectral, ax=ax, vmax=80)
#cb = plt.colorbar(pm, shrink=0.5)
wradlib.vis.add_lines(ax, borders, color='black', lw=0.3)
#for i in range(len(gauges)):
#    color = "red"
#    if gauges.veg[i]=="geprüft":
#        color="white"
#    plt.text(gauges.x[i], gauges.y[i], "%d" % gageprec[i],
#             horizontalalignment='center', verticalalignment='center', color=color, fontsize=12)
plt.grid()
plt.xlabel("UTM Easting")
plt.ylabel("UTM Northing")
plt.xlim(bbox[0], bbox[1])
plt.ylim(bbox[2], bbox[3])


ax = fig.add_subplot(133, aspect="equal")
pm = plt.pcolormesh(xrw, yrw, np.ma.masked_invalid(rw.sum(axis=0)), cmap=plt.cm.nipy_spectral, vmax=80)
#plt.colorbar(pm, shrink=0.75)
wradlib.vis.add_lines(ax, borders, color='black', lw=0.3)
plt.grid()
plt.xlabel("UTM Easting")
plt.ylabel("UTM Northing")
plt.xlim(bbox[0], bbox[1])
plt.ylim(bbox[2], bbox[3])

#plt.tight_layout()

#fig.subplots_adjust(bottom=0.01)
cbar_ax = fig.add_axes([0.2, 0.15, 0.6, 0.03])
fig.colorbar(pm, cax=cbar_ax, orientation="horizontal")


plt.tight_layout()

In [ ]:
# compute the KDTree
tree = KDTree(list(zip(xrw.ravel(), yrw.ravel())))
# query the tree for nearest neighbours
distances, rwix = tree.query(list(zip(gauges.x, gauges.y)), k=9)

In [ ]:
rwprec = np.median([rw.sum(axis=0).ravel()[ix_] for ix_ in rwix], axis=1)

In [ ]:
maxval = 70
goodix = np.array(gauges.veg=="geprüft")
fig = plt.figure(figsize=(12,6))
ax = plt.subplot(131, aspect="equal")
plt.plot([-10, 100], [-10, 100], "k--")
plt.plot(gageprec, radarprec, "k+")
plt.plot(gageprec[goodix], radarprec[goodix], "g+")
plt.xlim(0,maxval)
plt.ylim(0,maxval)
plt.xlabel("Gages (mm)")
plt.ylabel("Radar (mm)")
plt.grid()
plt.title("DX")

ax = plt.subplot(132, aspect="equal")
plt.plot([-10, 100], [-10, 100], "k--")
plt.plot(gageprec, radarprec2, "k+")
plt.plot(gageprec[goodix], radarprec2[goodix], "g+")
plt.xlim(0,maxval)
plt.ylim(0,maxval)
plt.xlabel("Gages (mm)")
plt.ylabel("Radar (mm)")
plt.grid()
plt.title("DX, att. corr.")

ax = plt.subplot(133, aspect="equal")
plt.plot([-10, 100], [-10, 100], "k--")
plt.plot(gageprec, rwprec, "k+")
plt.plot(gageprec[goodix], rwprec[goodix], "g+")
plt.xlim(0,maxval)
plt.ylim(0,maxval)
plt.xlabel("Gages (mm)")
plt.ylabel("Radar (mm)")
plt.grid()
plt.title("RW")

### Kumulativer Niederschlag über die Zeit

In [ ]:
dxseries = np.median(prec.reshape((len(dtimes), -1))[:,dxix], axis=2)
attcorrseries = np.median(prec2.reshape((len(dtimes), -1))[:,dxix], axis=2)
rwseries = np.median(rw.reshape((len(dtimes_rw), -1))[:,rwix], axis=2)

In [ ]:
# RW
dfrw = pd.DataFrame(rwseries, index=dtimes_rw)
dfrw = dfrw.cumsum()
# DX
dfdx = pd.DataFrame(dxseries, index=dtimes)
dfdx = dfdx.cumsum()
# Attenutation corrected
dfattcorr = pd.DataFrame(attcorrseries, index=dtimes)
dfattcorr = dfattcorr.cumsum()


In [ ]:
tmp = gdata.loc[tstart:tend] - gdata.loc[tstart]

In [ ]:
gix = np.where(np.array(gauges.veg)=="geprüft")[0]

In [ ]:
fig = plt.figure(figsize=(12,12))
for pi, i in enumerate(gix):
    plt.subplot(4,4,pi+1)
    dfrw[i].plot(label="RW")
    dfdx[i].plot(label="DX")
    dfattcorr[i].plot(label="DX corr.")
    tmp[i].plot(label="Gauge", color="black")
    plt.ylim(0,60)
    plt.grid()
    plt.title(gauges.name[i])
    if pi==12:
        #plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.legend(ncol=2)
plt.tight_layout()

In [ ]:
rwseries.shape

In [ ]:
dfrw